In [1]:
!pip install pyspark
!pip install py4j
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.apache.org/dyn/closer.lua/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark
from pyspark.context import SparkContext as sc
import pyspark.sql
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col 
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import min, max, mean

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 52.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=8e953728a1529949d49cb001de5eee31d961d91bcfc3b5a1a7b11842d4324863
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import files
demo = files.upload()

Saving users_1.csv to users_1.csv


In [98]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[2]") \
.appName("DEMO")  \
.config('spark.ui.port','4050') \
.getOrCreate()
spark

QUES_1 :  Create a RDD using the User.csv file

          i. Convert the RDD into Dataframe with an schema (Sno,Gender,Movie_id,rating,watch_count)

          ii. Remove the duplicate from the dataframe
          
          iii. Apply the group the movie_id and create a new column with an count_no

In [133]:
rdd = spark.sparkContext.textFile("users_1.csv")

In [134]:
rddCollect = rdd.collect()
print(rddCollect)

['2022', 'movie user list', 'process�', '', '1::F::1::10::48067', '2::M::56::16::70072', '3::M::25::15::55117', '4::M::45::7::02460', '5::M::25::20::55455', '6::F::50::9::55117', '7::M::35::1::06810', '8::M::25::12::11413', '9::M::25::17::61614', '10::F::35::1::95370', '11::F::25::1::04093', '12::M::25::12::32793', '13::M::45::1::93304', '14::M::35::0::60126', '15::M::25::7::22903', '16::F::35::0::20670', '17::M::50::1::95350', '18::F::18::3::95825', '19::M::1::10::48073', '20::M::25::14::55113', '21::M::18::16::99353', '22::M::18::15::53706', '23::M::35::0::90049', '24::F::25::7::10023', '25::M::18::4::01609', '26::M::25::7::23112', '27::M::25::11::19130', '28::F::25::1::14607', '29::M::35::7::33407', '30::F::35::7::19143', '31::M::56::7::06840', '32::F::25::0::19355', '33::M::45::3::55421', '34::F::18::0::02135', '35::M::45::1::02482', '36::M::25::3::94123', '37::F::25::9::66212', '38::F::18::4::02215', '39::M::18::4::61820', '40::M::45::0::10543', '41::F::18::4::15116', '42::M::25::

In [146]:
rdd=spark.sparkContext.parallelize(rddCollect)



In [149]:
rdd1 = rdd.zipWithIndex().filter(lambda a : a[1] > 3 )
type(rdd1)


pyspark.rdd.PipelinedRDD

In [150]:
rdd2 = rdd1.map(lambda a : a[0].split('::'))

In [151]:
col_name = rdd2.collect()[0]
rdd3 = rdd2.filter(lambda a : a != col_name)
df = rdd3.toDF(col_name)
df.show()



+---+---+---+---+-----+
|  1|  F|  1| 10|48067|
+---+---+---+---+-----+
|  2|  M| 56| 16|70072|
|  3|  M| 25| 15|55117|
|  4|  M| 45|  7|02460|
|  5|  M| 25| 20|55455|
|  6|  F| 50|  9|55117|
|  7|  M| 35|  1|06810|
|  8|  M| 25| 12|11413|
|  9|  M| 25| 17|61614|
| 10|  F| 35|  1|95370|
| 11|  F| 25|  1|04093|
| 12|  M| 25| 12|32793|
| 13|  M| 45|  1|93304|
| 14|  M| 35|  0|60126|
| 15|  M| 25|  7|22903|
| 16|  F| 35|  0|20670|
| 17|  M| 50|  1|95350|
| 18|  F| 18|  3|95825|
| 19|  M|  1| 10|48073|
| 20|  M| 25| 14|55113|
| 21|  M| 18| 16|99353|
+---+---+---+---+-----+
only showing top 20 rows



In [152]:

from pyspark.context import SparkContext as sc
deptcolumn = ["Sno","Gender","Movie_id","rating","watch_count"]

df3 = rdd.createDataFrame(rdd,deptcolumn)


In [157]:
schema=StructType([\
                  StructField("Sno",IntegerType(),True),\
                  StructField("Gender",StringType(),True),\
                  StructField("Movie_id",IntegerType(),True),\
                  StructField("rating", IntegerType(),True),\
                  StructField("watch_count",IntegerType(),True),
                   ])


In [164]:
dff=spark.read.csv("users_1.csv", sep="::",schema=schema)
dff.show(truncate = False)


+----+------+--------+------+-----------+
|Sno |Gender|Movie_id|rating|watch_count|
+----+------+--------+------+-----------+
|2022|null  |null    |null  |null       |
|null|null  |null    |null  |null       |
|null|null  |null    |null  |null       |
|1   |F     |1       |10    |48067      |
|2   |M     |56      |16    |70072      |
|3   |M     |25      |15    |55117      |
|4   |M     |45      |7     |2460       |
|5   |M     |25      |20    |55455      |
|6   |F     |50      |9     |55117      |
|7   |M     |35      |1     |6810       |
|8   |M     |25      |12    |11413      |
|9   |M     |25      |17    |61614      |
|10  |F     |35      |1     |95370      |
|11  |F     |25      |1     |4093       |
|12  |M     |25      |12    |32793      |
|13  |M     |45      |1     |93304      |
|14  |M     |35      |0     |60126      |
|15  |M     |25      |7     |22903      |
|16  |F     |35      |0     |20670      |
|17  |M     |50      |1     |95350      |
+----+------+--------+------+-----

In [167]:
dataframe = dff.na.drop()
dataframe.show()

+---+------+--------+------+-----------+
|Sno|Gender|Movie_id|rating|watch_count|
+---+------+--------+------+-----------+
|  1|     F|       1|    10|      48067|
|  2|     M|      56|    16|      70072|
|  3|     M|      25|    15|      55117|
|  4|     M|      45|     7|       2460|
|  5|     M|      25|    20|      55455|
|  6|     F|      50|     9|      55117|
|  7|     M|      35|     1|       6810|
|  8|     M|      25|    12|      11413|
|  9|     M|      25|    17|      61614|
| 10|     F|      35|     1|      95370|
| 11|     F|      25|     1|       4093|
| 12|     M|      25|    12|      32793|
| 13|     M|      45|     1|      93304|
| 14|     M|      35|     0|      60126|
| 15|     M|      25|     7|      22903|
| 16|     F|      35|     0|      20670|
| 17|     M|      50|     1|      95350|
| 18|     F|      18|     3|      95825|
| 19|     M|       1|    10|      48073|
| 20|     M|      25|    14|      55113|
+---+------+--------+------+-----------+
only showing top

In [169]:
print('Remove the duplicate from the dataframe')
dataframe.distinct().show()

Remove the duplicate from the dataframe
+----+------+--------+------+-----------+
| Sno|Gender|Movie_id|rating|watch_count|
+----+------+--------+------+-----------+
|  53|     M|      25|     0|      96931|
| 181|     M|      18|    17|      33186|
| 221|     F|      25|     0|      94063|
| 369|     M|      35|     1|      55110|
| 632|     M|      45|    17|       7649|
| 718|     M|      18|     2|      90211|
| 725|     M|      56|    11|       8904|
|1206|     F|      50|     7|      33764|
|1575|     F|      50|     7|      94949|
|1582|     M|      18|     0|      10705|
|2191|     M|      18|     4|      45231|
|2424|     M|      25|    17|      95035|
|2487|     M|      45|     7|      55438|
|2493|     M|      45|     2|      90631|
|2818|     M|      25|    12|      55106|
|2829|     M|      45|    20|      10013|
|3042|     M|      56|     1|      49042|
|3371|     M|      25|     2|      55405|
|3519|     F|      25|    14|      11215|
|3615|     M|      25|    17|      7

In [174]:
df3 = dff.filter("Sno >= 0 ").select(['Movie_id','Watch_count'])
df3.na.drop()
df3.show()

+--------+-----------+
|Movie_id|Watch_count|
+--------+-----------+
|    null|       null|
|       1|      48067|
|      56|      70072|
|      25|      55117|
|      45|       2460|
|      25|      55455|
|      50|      55117|
|      35|       6810|
|      25|      11413|
|      25|      61614|
|      35|      95370|
|      25|       4093|
|      25|      32793|
|      45|      93304|
|      35|      60126|
|      25|      22903|
|      35|      20670|
|      50|      95350|
|      18|      95825|
|       1|      48073|
+--------+-----------+
only showing top 20 rows



QUES 2 : convert the Dataframe into RDD 

In [175]:
rdd=dataframe.rdd

In [176]:
print(rdd.collect())

[Row(Sno=1, Gender='F', Movie_id=1, rating=10, watch_count=48067), Row(Sno=2, Gender='M', Movie_id=56, rating=16, watch_count=70072), Row(Sno=3, Gender='M', Movie_id=25, rating=15, watch_count=55117), Row(Sno=4, Gender='M', Movie_id=45, rating=7, watch_count=2460), Row(Sno=5, Gender='M', Movie_id=25, rating=20, watch_count=55455), Row(Sno=6, Gender='F', Movie_id=50, rating=9, watch_count=55117), Row(Sno=7, Gender='M', Movie_id=35, rating=1, watch_count=6810), Row(Sno=8, Gender='M', Movie_id=25, rating=12, watch_count=11413), Row(Sno=9, Gender='M', Movie_id=25, rating=17, watch_count=61614), Row(Sno=10, Gender='F', Movie_id=35, rating=1, watch_count=95370), Row(Sno=11, Gender='F', Movie_id=25, rating=1, watch_count=4093), Row(Sno=12, Gender='M', Movie_id=25, rating=12, watch_count=32793), Row(Sno=13, Gender='M', Movie_id=45, rating=1, watch_count=93304), Row(Sno=14, Gender='M', Movie_id=35, rating=0, watch_count=60126), Row(Sno=15, Gender='M', Movie_id=25, rating=7, watch_count=22903), 

QUES 3 : what is the usage of the lit()?

In [167]:
"""lit() functions of Pyspark SQL are used 
to add a new column to the Dataframe 
by putting a literal or constant value on it."""

QUES 4 : Difference between the group By and partition by?

In [167]:
"""1. the GROUP BY clause is used to group data using one or more columns which is Depending on the needs,
   2. Used with  functions like min, max, avg, sum, etc."""

"""
1. The Partition By clause is used to get the result set  is divided into partitions using  and each subset of the partitioned data is then subjected to computation.
2. It is Used tp partition computation on each subset of partitioned data .
"""




QUES 5 : Write an sql statement on the existing dataframe with an condition to fetch the maximum count_no.

In [177]:
dataframe.createOrReplaceTempView("ME")
spark.sql("SELECT COUNT(*) FROM ME").show()
spark.sql("SELECT COUNT(distinct Watch_count) FROM ME").show()

+--------+
|count(1)|
+--------+
|    5974|
+--------+

+---------------------------+
|count(DISTINCT Watch_count)|
+---------------------------+
|                       3374|
+---------------------------+



QUES 6 : reate a temporary table from the Existing dataframe

I. Perform the self join on the temporary table

II. Perform the union operation on the temporary table (you can use same table twice)

III. Delete the temporary table

In [178]:
dataframe.registerTempTable('ALTENRV')
sqlContext = SQLContext(spark)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [180]:
twice = spark.sql("SELECT * FROM ALTENRV e LEFT OUTER JOIN ALTENRV d ON e.S_no == d.watch_count")
twice.show(truncate=False)

AnalysisException: ignored

In [74]:


spark.sql("select * from PritamTable union select * from PritamTable").show()

In [75]:


spark.catalog.dropTempView("PritamTable")

+--------------------+
|                 _c0|
+--------------------+
|                2022|
|     movie user list|
|            process�|
|  1::F::1::10::48067|
| 2::M::56::16::70072|
| 3::M::25::15::55117|
|  4::M::45::7::02460|
| 5::M::25::20::55455|
|  6::F::50::9::55117|
|  7::M::35::1::06810|
| 8::M::25::12::11413|
| 9::M::25::17::61614|
| 10::F::35::1::95370|
| 11::F::25::1::04093|
|12::M::25::12::32793|
| 13::M::45::1::93304|
| 14::M::35::0::60126|
| 15::M::25::7::22903|
| 16::F::35::0::20670|
| 17::M::50::1::95350|
+--------------------+
only showing top 20 rows



QUES 7 : PySpark dataframe can be converted to pandas dataframe using the function toPandas() method

In [181]:
Newdataframe = dataframe.toPandas()
print(Newdataframe)

       Sno Gender  Movie_id  rating  watch_count
0        1      F         1      10        48067
1        2      M        56      16        70072
2        3      M        25      15        55117
3        4      M        45       7         2460
4        5      M        25      20        55455
...    ...    ...       ...     ...          ...
5969  6036      F        25      15        32603
5970  6037      F        45       1        76006
5971  6038      F        56       1        14706
5972  6039      F        45       0         1060
5973  6040      M        25       6        11106

[5974 rows x 5 columns]
